In [1]:
import numpy as np
from scipy.optimize import differential_evolution
from scipy.special import expit
from sklearn.metrics import roc_auc_score


class Boosting_Elcl:
    def __init__(self, tol=0.4, max_iter=100):
        self.tol = tol
        self.max_iter = max_iter
        pass
    def fit(self, X_train, y_train, max_elems):
        self.max_elems = max_elems
        
        def W(x):
            y2, y1, sigma = x[0], x[1], x[2:]
            res = np.prod(X_train <= sigma[None, :], axis=1)
            ans = res
            ans[res == 1] = y1
            ans[res == 0] = y2    
            return np.sum((ans - target) ** 2)

                
        self.sigmas = []
        target = y_train.copy()
        # меняем обозначение класса с 0 на -1
        target[target == 0] = -1
        
        #границы для y1, y2
        bounds = [(-1, 1), (-1, 1)]
        #границы для значений сигма
        bounds.extend([(0, i) for i in max_elems])
        
        
        ans = np.zeros(X_train.shape[0])
        
        for _ in range(self.max_iter):
            #Подбор нового эл.кл.
            tmp = differential_evolution(W, bounds).x
            self.sigmas.append((tmp[0], tmp[1], tmp[2:]))
            
            #Получение предсказания композиции уже подобранных эл.кл.
            sigma = tmp[2:]
            res = np.prod(X_train <= sigma[None, :], axis=1)
            ans[res == 1] += tmp[1]
            ans[res == 0] += tmp[0]
            prob = 1 / (1 + np.exp(-ans))
            loss = np.mean(np.logaddexp(0, -y_train * ans))
            if loss < self.tol:
                break
            
            #Вычисление нового target
            target = y_train * expit(-y_train * ans)
            
            
    def predict(self, X_test):
        ans = np.zeros(X_test.shape[0])
        
        for y2, y1, sigma in self.sigmas:
            res = np.prod(X_test <= sigma[None, :], axis=1)
            ans[res == 1] += y1
            ans[res == 0] += y2
            
        #Вероятность класса 1
        proba = expit(ans)[:, np.newaxis]
        #np.hstack((proba, 1 - proba))
        
        # Получаем метки
        tmp = np.sign(ans)
        tmp[tmp == -1] = 0
        tmp = tmp.astype(np.int)
        
        return proba
        

In [2]:
class One_vs_all:
    def __init__(self):
        pass
    def fit(self, X, y, max_elems):
        self.classes = np.unique(y)
        self.models = []
        for k in self.classes:
            tmp_model = Boosting_Elcl()
            y_tmp = y.copy()
            y_tmp[y == k] = 1
            y_tmp[y != k] = -1
            tmp_model.fit(X, y_tmp, max_elems)
            self.models.append(tmp_model)
            
    def predict(self, X):
        res = np.zeros((X.shape[0], len(self.classes)))
        for k, model in enumerate(self.models):
            res[:, k] = model.predict(X)[:, 0]
            
        return np.argmax(res, axis=1)
            

In [3]:
from sklearn.datasets import make_classification

max_el = 17

X, y = make_classification(n_samples=40, n_features=5, n_informative=3, n_redundant=2, n_classes=4) 
max_elems = np.ones(X.shape[1], dtype=np.int) * max_el
X = (X % max_el).astype(int)

In [6]:
m = One_vs_all()
m.fit(X, y, max_elems)
ans = m.predict(X)

In [7]:
def ohe(target, classes):
    return (classes == target[:, None]).astype(int)


classes = np.arange(len(np.unique(y)))
tmp =  ohe(y, classes)

roc_auc_score(tmp, ohe(ans, classes), average='macro')

0.5